<a href="https://colab.research.google.com/github/DavidSlva/Comunicaciones-Digitales-Laboratorio/blob/master/encuentra_tu_casa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install sklearn
!pip install gradio

In [ ]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr



GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_key = open("key.txt", "r").read().strip("\n")


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [ ]:
functions = [
    {
        # "name": "get_hauses",
        # "description": """Utiliza esta función para que el cliente obtenga todas las casas disponibles""",
        # "parameters": {
        #     "type": "object",
        #     "properties": {
        #         "query": {
        #             "type": "string",
        #             "description": f"""
        #                     User query in JSON. Responses should be summarized and should include the article URL reference
        #                     """,
        #         }
        #     },
        #     "required": ["query"],
        # },
        "name": "get_bests_hauses",
        "description": """Utiliza esta función para que el cliente tenga las casas que mejor se adapten a su petición""",
        "parameters": {
            "type": "object",
            "properties": {
                "description": {
                    "type": "string",
                    "description": f"""Descripción del usuario en texto plano basado en lo que quiere y en los gustos del usuario.""",
                }
            },
            "required": ["description"],
        },
    }
]

In [ ]:
with open('casas.json', 'r') as file:
    casas = json.load(file)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response
def get_best_descriptions(description):
    response_referent = embedding_request(description)
    print(description)
    embedding_referent = response_referent["data"][0]["embedding"]
    descripciones_json = [item['descripcion'] for item in casas]
    embeddings_results = []
    for descripcion_casa in descripciones_json:
      response_casa = embedding_request(descripcion_casa)
      embedding_casa = response_casa["data"][0]["embedding"]
      embeddings_results.append(embedding_casa)
    # Calcular la similitud coseno entre el embedding de la descripción de referencia y los embeddings del JSON
    similarity_scores = cosine_similarity([embedding_referent], embeddings_results)[0]

    # # Ordenar las descripciones del JSON por similitud en orden descendente
    descripciones_ordenadas = [(casas[i]['descripcion'], similarity_scores[i]) for i in range(len(similarity_scores))]
    descripciones_ordenadas.sort(key=lambda x: x[1], reverse=True)
    return descripciones_ordenadas


In [ ]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_functions(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()

def call_functions(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message["message"]["function_call"]["name"] == "get_bests_hauses":
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        results = get_best_descriptions(parsed_output["description"])
        messages.append({
            "role": "function",
            "name": full_message["message"]["function_call"]["name"],
            # "content": str(results),
            "content": str(results),
        })
        # print("Got search results, summarizing content")
        response = chat_completion_request(messages)
        return response.json()
    else:
        raise Exception("Function does not exist and cannot be called")

In [ ]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [ ]:
paper_system_message = """Tu nombre es Home Search Inteligence. Eres el encargado de encontrar las mejores casas en base a la descripción que
te brinde un interesado. Debes ser cordial y adaptarte a las emociones que detectes en el
 usuario. Es importante señalar también que solo puedes ayudar a buscar casas por la descripción de la misma,
 Si necesitan algo por precio no los puedes ayudar con eso. Es súper importante también, que siempre señales en qué puedes ayudar a la persona, que tenga clara tu función."""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

In [ ]:
def chatbot_response(user_input, history):
    history = history or []
    paper_conversation.add_message("user", user_input)
    chat_response = chat_completion_with_function_execution(
        paper_conversation.conversation_history, functions=functions
    )
    assistant_message = chat_response["choices"][0]["message"]["content"]
    paper_conversation.add_message("assistant", assistant_message)
    history.append((user_input, assistant_message))
    return history,history

In [ ]:
chatbot = gr.Chatbot(color_map=("green", "pink"))
iface = gr.Interface(
    chatbot_response,    # La función que maneja las respuestas del chatbot
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
    title="Home Search Inteligence"
)
iface.launch(share=True)


<ipython-input-42-57d0e2d766c1>:1: GradioDeprecationWarning: The 'color_map' parameter has been deprecated.
  chatbot = gr.Chatbot(color_map=("green", "pink"))


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4ead394c2ced0b659b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
